In [1]:
import nltk
from datasets import load_dataset
# import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

2023-11-20 12:57:14.986863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 12:57:18.731207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-20 12:57:18.731278: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-20 12:57:28.516614: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
!pip install evaluate
!pip install accelerate -U
!pip install datasets
!pip install accelerate -U

Requirement already up-to-date: accelerate in /home/shiridi/.local/lib/python3.8/site-packages (0.24.1)
Requirement already up-to-date: accelerate in /home/shiridi/.local/lib/python3.8/site-packages (0.24.1)


In [3]:
import evaluate

In [4]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

zsh:1: no matches found: transformers[sentencepiece]


In [5]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
dataset = load_dataset("toughdata/quora-question-answer-dataset")
dataset = dataset["train"].train_test_split(test_size=0.2)

In [7]:
def preprocess_labels(examples,maxl=128):
    # print(examples,"***")
    if(str(examples)=="nan"):
        examples="none"

    labels = tokenizer(text_target=examples, max_length=maxl, truncation=True,padding="max_length")


    return labels["input_ids"]

def preprocess_function(examples,maxl=512):
    model_inputs = tokenizer(examples, max_length=maxl, truncation=True,padding="max_length")
    # labels = tokenizer(text_target=examples["answer"], max_length=128, truncation=True,padding="max_length")
    # model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs["input_ids"]


In [8]:
import pandas as pd
df=pd.read_csv("../Squad_PreProcessed.csv")
passage_ids=pd.read_csv("../passage_ids.csv")
passage_ids.set_index("id",inplace=True)
df["passage"]=df["passage_id"].apply(lambda x: passage_ids.loc[x]["passage"])

In [126]:
df

,qn_id,title,passage_id,qn_id.1,question,answer_text,answer_start,CorrectSentence,passage,input
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s,269,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,answer the question: Beyoncé Giselle Knowles-C...
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing,207,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,answer the question: Beyoncé Giselle Knowles-C...
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003,526,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,answer the question: Beyoncé Giselle Knowles-C...
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé_0,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas",166,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,answer the question: Beyoncé Giselle Knowles-C...
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé_0,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s,276,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,answer the question: Beyoncé Giselle Knowles-C...
...,...,...,...,...,...,...,...,...,...,...
86602,5735d259012e2f140011a09d,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Oregon,229,1,"Kathmandu Metropolitan City (KMC), in order to...",answer the question: Kathmandu Metropolitan Ci...
86603,5735d259012e2f140011a09e,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09e,What was Yangon previously known as?,Rangoon,414,2,"Kathmandu Metropolitan City (KMC), in order to...",answer the question: Kathmandu Metropolitan Ci...
86604,5735d259012e2f140011a09f,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Minsk,476,2,"Kathmandu Metropolitan City (KMC), in order to...",answer the question: Kathmandu Metropolitan Ci...
86605,5735d259012e2f140011a0a0,Kathmandu,Kathmandu_18979,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,1975,199,1,"Kathmandu Metropolitan City (KMC), in order to...",answer the question: Kathmandu Metropolitan Ci...


In [9]:
df["input"]=df["passage"].apply(lambda x : "answer the question: "+x)
df["temp"]=df["question"].apply(lambda x: " <sep> "+ str(x))
df["input"]=df["input"]+df["temp"]
df["input_ids"]=df["input"].apply(lambda x: preprocess_function(x,512))

In [10]:
lbls=[None for i in range(len(df))]
for i in range(len(df)):
    lbls[i]=preprocess_labels(df["answer_text"][i],128)

In [11]:
df["labels"]=lbls

In [12]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [13]:
ind=int(0.8*len(df))
train=df.iloc[:ind,:]
test=df.iloc[ind:,]

In [17]:
train=train[["input_ids","labels"]]
test=test[["input_ids","labels"]]

In [20]:
from torch.utils.data import Dataset, DataLoader


In [21]:
squad_dataset = load_dataset("squad")
train_dataset = squad_dataset["train"]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [22]:
class SquadDataset(Dataset):
    def __init__(self, squad_examples, tokenizer):
        self.squad_examples = squad_examples
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.squad_examples)

    def __getitem__(self, idx):
        example = self.squad_examples[idx]

        # Tokenize the context and question
        inputs = self.tokenizer(
            example["context"],
            example["question"],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512,  # Adjust as needed based on your model's maximum input length
        )

        # Tokenize the answer span
        answer_start = example["answers"]["answer_start"][0]
        answer_end = answer_start + len(example["answers"]["text"][0])
        labels = tokenizer(
            example["answers"]["text"][0],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=32,  # Adjust as needed based on your maximum answer length
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
        }

custom_dataset = SquadDataset(train_dataset, tokenizer)

In [24]:
train_dataloader = DataLoader(custom_dataset, batch_size=8, shuffle=True)

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=custom_dataset,
    # eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

  0%|          | 0/21900 [00:00<?, ?it/s]